In [1]:
import datetime
from bs4 import BeautifulSoup
import requests
import json
import time
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.by import By
from tqdm import tqdm
import numpy as np
import pandas as pd

yesterday = datetime.datetime.now() - datetime.timedelta(1)
yesterday_str = datetime.datetime.strftime(yesterday, '%Y%m%d')

# Get HK

In [2]:
# Get English Name of stock
ccass_str_lst_url_en = 'https://www3.hkexnews.hk/sdw/search/ccass_stock_list.htm?sortby=stockcode&shareholdingdate='+yesterday_str

browser = webdriver.Chrome()
browser.get(ccass_str_lst_url_en)

time.sleep(2)
table_row_cnt = browser.find_element(By.XPATH, "//table[@class='table table-list']/tbody")
table_str = table_row_cnt.text
stock_lst = table_str.split("\n")

stock_code_lst = []
individual_stock = []
group_stock_lst = []
for i in range(len(stock_lst)):
    stock_code = stock_lst[i][:5]
    stock_name = stock_lst[i][6:]
    stock_code_lst.append(stock_code)
    individual_stock.append(stock_code)
    individual_stock.append(stock_name)
    group_stock_lst.append(individual_stock)
    individual_stock = []

stock_lst_df_en = pd.DataFrame(group_stock_lst)
stock_lst_df_en.columns = ['stock_code','stock_name_en']

browser.close()

In [ ]:
hk_stock_code_lst = stock_lst_df_en['stock_code'].tolist()

browser = webdriver.Chrome()

hk_stock_profile_lst = []
for i in tqdm(hk_stock_code_lst):
    stock_profile = []
    stock_code = i.lstrip('0')
    hkex_company = 'https://www.hkex.com.hk/Market-Data/Securities-Prices/Equities/Equities-Quote?sym='+stock_code+'&sc_lang=en'
    
    browser.get(hkex_company)
    
    time.sleep(2)
    
    try:
        mkt_cap = browser.find_element(By.XPATH, '//dt[@class="ico_data col_mktcap"]')
        cap = mkt_cap.text
        
        lot_size = browser.find_element(By.XPATH, '//dt[@class="ico_data col_lotsize"]')
        lot = lot_size.text

        sector_industry = browser.find_element(By.XPATH, '//span[@class="col_industry_hsic"]')
        sector = sector_industry.text

        comp_profile = browser.find_element(By.XPATH, '//div[@class="company_txt col_summary"]')
        profile = comp_profile.text

        stock_profile.append(i)
        stock_profile.append(cap)
        stock_profile.append(lot)
        stock_profile.append(sector)
        stock_profile.append(profile)
        hk_stock_profile_lst.append(stock_profile)
    except:
        try:
            mkt_cap = browser.find_element(By.XPATH, '//dt[@class="ico_data col_mktcap tooltip"]')
            cap = mkt_cap.text
            
            lot_size = browser.find_element(By.XPATH, '//dt[@class="ico_data col_lotsize"]')
            lot = lot_size.text

            sector_industry = browser.find_element(By.XPATH, '//span[@class="col_industry_hsic"]')
            sector = sector_industry.text

            comp_profile = browser.find_element(By.XPATH, '//div[@class="company_txt col_summary"]')
            profile = comp_profile.text

            stock_profile.append(i)
            stock_profile.append(cap)
            stock_profile.append(lot)
            stock_profile.append(sector)
            stock_profile.append(profile)
            hk_stock_profile_lst.append(stock_profile)
        except:
            continue

browser.close()

 32%|███████████▎                       | 1759/5449 [1:59:50<3:57:49,  3.87s/it]

In [ ]:
hk_stock_profile_df = pd.DataFrame(hk_stock_profile_lst,columns = ['stock_code','market_cap','lot_size','sector_industry','profile'])

hk_stock_profile_df = hk_stock_profile_df.replace(r'^\s*$', np.nan, regex=True)
hk_stock_profile_df.dropna(inplace=True)
hk_stock_profile_df = hk_stock_profile_df.iloc[:-1]
hk_stock_profile_df.reset_index(drop=True,inplace=True)

mkt_cap_lst = []
for mc in hk_stock_profile_df['market_cap'].tolist():
    if ('HK$' in mc) == True:
        cap = mc.replace('HK$','')
        cap = cap.replace(',','')
        if ('B' in cap) == True:
            full_cap = float(cap[:-1])* 1000000000
            mkt_cap_lst.append(full_cap)
        if ('M' in cap) ==True:
            full_cap = float(cap[:-1])* 1000000
            mkt_cap_lst.append(full_cap)
    else:
        mkt_cap_lst.append(np.nan)
        
hk_stock_profile_df['market_cap_long'] = mkt_cap_lst

hk_stock_profile_df.dropna(inplace=True)
hk_stock_profile_df.reset_index(drop=True,inplace=True)
hk_stock_profile_df[['sector','sub_sector','industry','extra']] = hk_stock_profile_df['sector_industry'].str.split('-',expand=True)
hk_stock_profile_df.fillna('',inplace=True)
hk_stock_profile_df['industry'] = hk_stock_profile_df["industry"].astype(str) + hk_stock_profile_df["extra"]
hk_stock_profile_df.drop('extra',axis=1,inplace=True)

hk_stock_profile_df.head()

In [ ]:
hk_stock_profile_df.to_csv('hk_stock_profile.csv',index=True)

# Get US

In [ ]:
def get_us_data():
    # Fetch data from API
    url = "https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=0&offset=0&download=true"

    headers = {'user-agent':'my-app/0.0.1'}
    
    # To json
    response = requests.get(url, headers = headers, timeout = None)
    data = response.text
    parse_json = json.loads(data)
    row = parse_json["data"]["rows"]
    
    # To dataframe
    stk = pd.DataFrame(row)
    df = stk[['symbol','sector','industry','marketCap']]
    df.columns = ['stk_code','sector','industry','mkt_cap']
    
    df.replace('',np.nan,inplace=True)
    df['mkt_cap'] = df['mkt_cap'].astype(float)
    
    return df

In [ ]:
def process_us_data():
    # Collect data
    df = get_us_data()
    
    mkt_cap_lst = df['mkt_cap'].values.tolist()
    
    mkt_cap_gp = []
    for i in range(len(mkt_cap_lst)):
        cap = mkt_cap_lst[i]
        if cap > 200000000000:
            mkt_cap_gp.append('mega')
        elif cap > 10000000000 and cap <= 200000000000:
            mkt_cap_gp.append('large')
        elif cap > 2000000000 and cap <= 10000000000:
            mkt_cap_gp.append('medium')
        elif cap > 300000000 and cap <= 2000000000:
            mkt_cap_gp.append('small')
        elif cap > 50000000 and cap <= 300000000:
            mkt_cap_gp.append('micro')
        elif cap <= 50000000:
            mkt_cap_gp.append('nano')
        else:
            mkt_cap_gp.append(np.nan)
            
    df['mkt_cap_gp'] = mkt_cap_gp
    
    return df

In [ ]:
us = process_us_data()

In [ ]:
us.to_csv('all_us_stk.csv',index=False)